# Denoising del Dataset JUMP Cell con Noise2Void

Il denoising di immagini microscopiche rappresenta una sfida affascinante nel campo dell’elaborazione delle immagini biologiche. Spesso, le immagini raccolte dai microscopi contengono rumore che può compromettere l’analisi successiva, e ottenere una **Ground Truth** perfetta è praticamente impossibile.  

In questo notebook esploreremo come utilizzare una **rete neurale U-Net**, combinata con l’algoritmo **Noise2Void (N2V)**, per ripulire le immagini del **Dataset JUMP**, un set di immagini mediche microscopiche. Grazie a Noise2Void, è possibile estrarre immagini “pulite” senza bisogno di dati di riferimento, rendendo l’approccio ideale per dataset complessi e realistici come questo.

Il nostro obiettivo è duplice:  
- Scoprire le potenzialità delle **Deep Neural Networks** nel ridurre il rumore delle immagini.  
- Confrontare i risultati ottenuti con approcci tradizionali di denoising, approfondendo al contempo i meccanismi interni di Noise2Void e il suo funzionamento.

Il **Dataset JUMP** offre una prospettiva unica: immagini microscopiche di cellule in cui il rumore è intrinseco e inevitabile. In questo contesto, Noise2Void si distingue per la sua capacità di generare immagini pulite e affidabili, aprendo la strada a nuove possibilità nell’analisi di dati biologici complessi.

## Download del Dataset

Il **Dataset JUMP** viene scaricato in formato **.TIFF Hyperstack** dalla piattaforma **Zenodo**. È composto da **517 immagini**, ciascuna organizzata su **4 canali**, per un totale di **2068 immagini** con dimensioni di **540x540 pixel**.  

I quattro canali separano le componenti **RGB e Alpha**. Questo è particolarmente utile nell’analisi cellulare, perché permette di distinguere con maggiore precisione le diverse parti della cellula e le loro colorazioni, facilitando la successiva elaborazione e visualizzazione.

Dopo il download, il dataset viene suddiviso in due insiemi principali: **Training Data** e **Validation Data**.  
- I **Training Data** vengono utilizzati dalla rete neurale per apprendere i valori ottimali dei suoi parametri.  
- I **Validation Data** servono invece a monitorare l’andamento dell’addestramento e prevenire fenomeni di **overfitting**, garantendo che la rete generalizzi bene su dati mai visti.

In questo progetto abbiamo scelto uno **split ratio di 0.8**, cioè l’**80% delle immagini** viene destinato al training, mentre il **20% rimanente** costituisce il validation set.


In [ ]:
import tifffile
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from careamics import CAREamist
from careamics.config import create_n2v_configuration
from careamics.utils.metrics import scale_invariant_psnr

sys.path.append('library')

import library.dataset as dataset

In [ ]:
#Download the Dataset
#await dataset.load_jump_dataset("notebooks/data/jump/noisy.tiff")

#Split the Dataset into Training and Validation
dataset.split_jump_dataset("notebooks/data/jump/noisy.tiff", split_ratio=0.8)

## Visualizzazione del Dataset

Per comprendere meglio la struttura del **Dataset JUMP**, iniziamo visualizzando alcune immagini a scopo illustrativo.  

Per ciascuna immagine vengono mostrati i **quattro canali** che la compongono, permettendo di osservare separatamente le componenti RGB e Alpha. Questo passaggio ci aiuta a familiarizzare con i dettagli visivi delle cellule e a capire come la rete neurale dovrà interpretare le diverse informazioni presenti in ciascun canale.


In [ ]:
train_images = tifffile.imread("notebooks/data/jump/noisy_train.tiff")
val_images = tifffile.imread("notebooks/data/jump/noisy_val.tiff")

starting_index = 0 #Change this to show different images

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

for idx in range(starting_index, starting_index + 4): #Couples the axes with the indexes
    for ax, ch in zip(axes[idx].flat, range(4)):
        ax.imshow(train_images[idx][ch], cmap="gray")
        ax.set_title(f"Training Image {idx} - Channel {ch+1}")
    
plt.tight_layout()

## Creazione della Configurazione di Training

Prima di avviare l’addestramento della rete neurale, è necessario definire un oggetto di **configurazione** che specifichi i parametri principali sia della rete stessa sia del processo di training.  

Tra i parametri più importanti troviamo:  

- **batch_size**: indica quante immagini vengono elaborate insieme in un singolo passo dell’addestramento.  
- **num_epochs**: definisce il numero di volte in cui l’intero dataset viene passato attraverso la rete durante l’addestramento.  

È fondamentale impostare questi valori in base alle **caratteristiche dell’hardware** a disposizione. Un batch troppo grande potrebbe esaurire la memoria della GPU, mentre un numero di epoche troppo basso potrebbe impedire alla rete di apprendere in modo efficace.

In [ ]:
config = create_n2v_configuration(
    experiment_name="jump_cells_n2v",
    data_type="array",
    axes="SCYX",
    n_channels=4,
    patch_size=(64, 64),
    batch_size=32,
    num_epochs=1,
    independent_channels=True,
)

print(config)

## Addestramento del Modello

L'addestramento del modello viene effettuato creando un oggetto **CAREamist** a partire dalla configurazione definita in precedenza.  

Il metodo `.train` del modello richiede semplicemente di fornire le **immagini di Training** e le **immagini di Validation**. Durante il processo di training, è possibile monitorare diversi parametri che ne descrivono l’andamento, come la perdita (loss) e altre metriche di performance, per valutare come la rete sta imparando dai dati.  

Al termine dell’addestramento, gli **stati migliori del modello** vengono salvati in file appositi con estensione **.ckpt (Checkpoint)**. Questi checkpoint permettono di riprendere il training in un secondo momento o di utilizzare direttamente il modello addestrato per il denoising delle immagini.


In [ ]:
# Before proceding, make sure your GPU is available to PyTorch or the training will be very slow

careamist = CAREamist(source=config, work_dir="notebooks/models/jump")

# train model
print(f"Training starting now...")
careamist.train(train_source=train_images, val_source=val_images)
print("Training ended!")

## Generazione delle Predizioni

Una volta completato l’addestramento, possiamo osservare come si comporta il modello chiedendogli di **predire le immagini pulite** a partire da alcuni campioni rumorosi.  

In generale, è buona pratica testare un modello su dati **diversi** da quelli utilizzati per l’addestramento. Tuttavia, nel caso dell’algoritmo **Noise2Void (N2V)** questa precauzione non è necessaria: l’algoritmo non utilizza mai le Ground Truth durante il training, quindi non ha "visto" le immagini in termini di valori ideali, anche se sono state impiegate per l’addestramento.  

Utilizzando il metodo `.predict` del modello, sfruttiamo l’**ultimo checkpoint disponibile** per generare le predizioni delle prime 10 immagini del dataset. Le immagini risultanti vengono poi salvate in memoria in formato **.TIFF**, pronte per essere analizzate e confrontate con le versioni rumorose originali.

In [ ]:
output_path = "notebooks/predictions/jump/predictions.tiff"
predict_counter = 10 # The number of images we want to predict

predictions = []
for i in range(predict_counter):
    print(f"Predicting batch number {i}")
    pred_batch = careamist.predict(source=train_images[i], data_type='array', axes='CYX')
    predictions.append(pred_batch)

predictions = np.concatenate(predictions, axis=0).squeeze()
os.makedirs(os.path.dirname(output_path), exist_ok=True)
tifffile.imwrite(output_path, predictions)
print(f"TIFF file saved to {output_path}")

## Visualizzazione delle Predizioni

A questo punto possiamo osservare i risultati delle predizioni generate dal modello.  

Per semplicità, mostriamo solo la **prima immagine** del dataset, visualizzandone separatamente i **quattro canali**. Questo permette di apprezzare con maggiore dettaglio come il modello abbia trattato le diverse componenti della cellula.

Inoltre, quando disponibili, confrontiamo le predizioni ottenute con un numero di epoche di **num_epochs=1** rispetto a **num_epochs=100**. Questo confronto evidenzia come l’addestramento più lungo influenzi la qualità delle immagini generate.

Le immagini risultanti rappresentano in modo chiaro sia le **potenzialità dell’algoritmo Noise2Void**, capace di rimuovere il rumore senza Ground Truth, sia i **limiti empirici** che emergono quando il rumore o le strutture complesse non possono essere completamente ricostruite.


In [ ]:
better_predictions_path = "notebooks/predictions/jump/predictions100.tiff"
if os.path.exists(better_predictions_path):
    better_predictions = tifffile.imread(better_predictions_path)
else:
    better_predictions = None

fig, ax = plt.subplots(4, 3 if better_predictions is not None else 2, figsize=(15, 20))
for i in range(4):
    ax[i, 0].imshow(train_images[0, i], cmap="gray")
    ax[i, 0].set_title(f"Noisy - Channel {i}")
    ax[i, 1].imshow(predictions[0].squeeze()[i], cmap="gray")
    ax[i, 1].set_title("Prediction 1 Epoch")
    if (better_predictions is not None):
        ax[i, 2].imshow(better_predictions[0].squeeze()[i], cmap="gray")
        ax[i, 2].set_title("Prediction 100 Epoch")